# Векторизация

## Про векторизацию
`NumPy` оптимизирован для работы с многомерными массивами, но циклы `python` нет. В связи с этим распространен подход, называемый **векторизацией**, при котором устраняются циклы, а вместо них вызываются встроенные в `NumPy` методы. Таким образом эти циклы осуществляются внутри `C` кода. 

Продемонстрируем это на примере вычисления суммы элементов массива `NumPy`. Ниже создаётся массив из $10^7$ случайных чисел. 

In [1]:
import numpy as np

N = 10 ** 7

x = np.random.random(N)

Измерим, сколько времени займёт вычисление суммы элементов этого массива в обычном цикле.

```{note}
[%%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time) --- [магическая команда](https://ipython.readthedocs.io/en/stable/interactive/magics.html#) `ipython`, которая измеряет время, затраченное на исполнение кода в ячейке, в которой она указана.
```

In [2]:
%%time
s1 = 0
for i in range(N):
    s1 += x[i]

CPU times: total: 2.94 s
Wall time: 3.2 s


Теперь используем для этого метод [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

```{note}
[%%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) --- то же самое, что и `%%time`, но время усредняется по нескольким запускам. 
```

In [3]:
%%timeit
s2 = x.sum()

15.1 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Видим значительную разницу в скорости циклов и векторизованного кода. 

У векторизации есть три основных плюса:
1. **Скорость**: векторизованный код исполняется гораздо быстрее, чем его аналог в циклах;
2. **Количество кода**: векторизованный код без циклов как правило короче, а значит в нем сложнее ошибиться.
3. **Выразительность**: векторизованный код больше похож на математическое выражение, что упрощает его чтение;

## Простые математические функции

В `NumPy` есть большое количество математических функций, полный список которых можно посмотреть [здесь](https://numpy.org/doc/stable/reference/routines.math.html). Применять их можно сразу к всему массиву. Тогда выбранная функция будет применена к всем элементам массива. Все циклы выполнятся в `C` коде.

In [3]:
%%time
y = np.zeros(N)
for i in range(N):
    y[i] = np.sin(x[i])

CPU times: total: 28.4 s
Wall time: 32.3 s


In [5]:
%%timeit
y = np.sin(x)

130 ms ± 4.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Агрегирующие функции

Так же в `NumPy` реализован ряд агрегирующих методов, таких как сумма элементов массива ([ndarray.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html#numpy.sum)), их произведение ([ndarray.prod](https://numpy.org/doc/stable/reference/generated/numpy.prod.html#numpy.prod)), минимум ([ndarray.min](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.min.html)), максимум ([ndarray.max](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.max.html)) и т.п. Такого рода операции часто применяются в статистике и доступные в `NumPy` функции такого рода перечисленны по [ссылке](https://numpy.org/doc/stable/reference/routines.statistics.html#averages-and-variances).

Продемонстрируем поведение таких функций на примере метода [numpy.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

In [12]:
matrix = np.array([
    [1, 2, 3],
    [10, 20, 30],
    [100, 200, 300]
])
print(matrix)

print(matrix.sum())

[[  1   2   3]
 [ 10  20  30]
 [100 200 300]]
666


По умолчанию все агрегирующие функции игнорируют размерность и форму массива и агрегируют все данные из массива в одно число. 

```{figure} /_static/lecture_specific/numpy/sum_1.svg
```

Каждая из них позволяет не только агрегировать по всему массиву целиком, но и вдоль обозначенной опциональным параметром `axis` оси. Например, чтобы просуммировать матрицу по столбцам, необходимо передать значение 0 в качестве параметра `axis`.

In [13]:
print(matrix.sum(axis=0))

[111 222 333]


```{figure} /_static/lecture_specific/numpy/sum_2.svg
```

А чтобы просуммировать по строкам --- значение 1.

In [15]:
print(x.sum(axis=1))

[15 15 15]


```{figure} /_static/lecture_specific/numpy/sum_3.svg
```

## Арифметика. 

Аналогично, всегда предпочтительнее использовать операторы `+`, `-`, `*`, `/`, `//`, `%`, `**`, `@`, `|`, `&` сразу к массивам целиком, а не к их элементам внутри циклов.

In [13]:
x = np.zeros(N)
y = np.ones(N)

In [14]:
%%time
z = np.zeros(N)
for i in range(N):
    z[i] = x[i] + y[i]

Wall time: 6.82 s


In [15]:
%%timeit
z = x + y

46.6 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Broadcasting

В ряде случаев допускается применять арифметические операторы к массивам разной формы. Строго говоря, `NumPy` может оперировать с двумя массивами, если их формы совместимы (`broadcastable`). Подробнее почитать о `broadcasting` в документации можно [здесь](https://numpy.org/doc/stable/user/theory.broadcasting.html#array-broadcasting-in-numpy) и [здесь](https://numpy.org/devdocs/user/basics.broadcasting.html).

**Пример 1. Скаляр и массив.** 

Начнем с самого простого примера. Если в арифметическом выражении участвует массив любой формы и скаляр, то можно представить, что скаляр расширяется до массива такой же формы, у которого все элементы равны значению скаляра.

```{figure} /_static/lecture_specific/vectorization/broadcasting_1.svg
```

На картинке выше массив `a` из трех элементов умножается на скаляр `b` (который можно представить в виде массива из одного элемента).

In [16]:
a = np.array([1, 2, 3])
b = np.array([2])
result = a * b
print(f"{a} * {b} = {result}")

[1 2 3] * [2] = [2 4 6]


```{note}
В реальности никакого промежуточного массива из двоек в примере выше не создается. Это лишь удобный способ представить себе, что происходит.
```

**Общий случай.** В более общем случае два массива совместимы по форме (`broadcastable`), если размеры вдоль всех их осей совместимы. Размеры вдоль двух осей совместимы, только если 
- они равны;
- один из них равен единице;

При этом проверка этих размеров происходит справа налево и форма массива меньшей размерности дополняется единицами слева до необходимого количества измерений.

Рассмотрим примеры совместимых и не совместимых форм.

Совместимые формы.

|`A.shape`|`B.shape`|`result.shape`|Комментарий|
| :---- | --- | --- | ---: |
| (3,) | (1, ) | (3, ) | Размер `B` вдоль единственной оси равен 1, формы `A` и `B` совместимы |
| (1, 3) | (3, 1) | (3, 3) | Размеры вдоль последней оси совместимы, т.к. размер `B` вдоль нее равен 1. Размеры вдоль первой оси совместимы, т.к. размер `A` вдоль неё равен 1|
|(8, 1, 6, 1)|(7, 1, 5)|(8, 7, 5, 6)|Массив `B` можно представить в виде массива с формой (1, 7, 1, 5). Размеры вдоль всех осей совместимы, т.к. вдоль каждой оси у одного из массивов длинна равняется 1|
|(15, 4, 3)|(15, 1, 3)|(15, 4, 3)| Размеры вдоль всех осей совместимы |

Несовместимые формы.

|`A.shape`|`B.shape`|Комментарий|
| :---- | --- | ---: |
|(3, )|(4, )| Размеры вдоль единственных осей не равны между собой, ни один из них не равен 1|
|(2, 1)| (2, 4, 3)| Сначала сравниваются 1 и 3: всё ок. Затем сравниваются 2 и 4: несовместимы. Можно представить форму (2, 1) в виде (1, 2, 1)|

**Пример 2. Матрица и вектор-строка**. 

Если складываются (умножаются и т.п.) двумерный и одномерный массивы, то операция возможна, если количество столбцов в матрице соответствует количеству элементов в одномерном массиве.

```{figure} /_static/lecture_specific/vectorization/broadcasting_2.svg
```

В примере выше складываются массивы с формами `(4, 3)` и `(3, )`, что эквивалентно записи `(4, 3)` и `(1, 3)`. Формы совместимы и результат выходит такой, будто одномерный массив расширяется до двумерного массива с формой `(4, 3)` "по столбцам".

In [17]:
a = np.array([[0, 0, 0], [10, 10, 10], [20, 20, 20], [30, 30, 30]])
b = np.array([0, 1, 2])
result = a + b
print(f"{a} \n + \n {b} \n = \n {result} ")

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]] 
 + 
 [0 1 2] 
 = 
 [[ 0  1  2]
 [10 11 12]
 [20 21 22]
 [30 31 32]] 


```{figure} /_static/lecture_specific/vectorization/broadcasting_3.svg
```

А в этом примере формы не совместимы.

In [18]:
a = np.array([[0, 0, 0], [10, 10, 10], [20, 20, 20], [30, 30, 30]])
b = np.array([1, 2, 3, 4])
a + b

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

**Пример 3. Матрица и вектор столбец.**

Аналогично можно сложить матрицу с вектором столбцом, но для этого этот массив нужно транспонировать, т.е. привести вектор формы `(n, )` (одномерный массив из `n` элементов) к форме `(n, 1)` (матрица из `n` строк длинны 1).

К сожалению, `ndarray.T` (`np.transpose`) не приведёт к желаемому результату, если массив одномерный. 

In [21]:
print(b, b.T)

[1 2 3 4] [1 2 3 4]


Сделать это можно изменив форму или "вставив ось".

In [25]:
n = b.shape[0]
print(b.reshape((n, 1)))
print()
print(b[:, np.newaxis])

[[1]
 [2]
 [3]
 [4]]

[[1]
 [2]
 [3]
 [4]]


Теперь можно сложить матрицу со столбцом.

In [28]:
result = a + b[:, np.newaxis]
print(result)

[[ 1  1  1]
 [12 12 12]
 [23 23 23]
 [34 34 34]]


**Пример 3. Вектор столбец и вектор строка**

Если складывать вектор столбец и вектор строку, то каждая из них как бы расширится до матрицы по строкам и по столбцам. 

![image](/_static/lecture_specific/vectorization/broadcasting_4.svg)


In [31]:
a = np.array([0, 10, 20, 30])[:, np.newaxis]
b = np.array([1, 2, 3])
result = a + b
print(f"{a} \n + \n {b} \n  = \n {result}")

[[ 0]
 [10]
 [20]
 [30]] 
 + 
 [1 2 3] 
  = 
 [[ 1  2  3]
 [11 12 13]
 [21 22 23]
 [31 32 33]]
